In [1]:
import pandas as pd
import numpy as np

# Задача 1

In [5]:
df = pd.read_csv('data-rodnik-neskuchniy-sad.csv') # загрузка данных
df.index = df['Показатель'] # сделаем колонку "Показатель" индексом таблицы
df.drop(axis=1, columns='Показатель', inplace=True)

def is_normal(x):
    if x['Результат анализа'] == 'б/цвета':
        return True
    elif 'в пределах' in x['Норматив']:
        try:
            left, right = ''.join([symb for symb in x['Норматив'] if not symb.isalpha()]).split('-')
            return float(left.replace(',', '.')) <= float(x['Результат анализа']) <= float(right.replace(',', '.'))
        except ValueError:
            return False
    elif 'не более' in x['Норматив']:
        try:
            if '-' in x['Норматив']:
                normal = float(x['Норматив'].split('-')[1].replace(',', '.'))
                curr_value = float(x['Результат анализа'])
                return curr_value <= normal

            right = ''.join([symb for symb in x['Норматив'] if not symb.isalpha()])
            return float(x['Результат анализа']) <= float(right.replace(',', '.'))
        except ValueError:
            return False
    return False

df['is_norm'] = df.apply(is_normal, axis=1)
df

,Единица измерений,Результат анализа,Норматив,is_norm
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,True
Запах,баллы,1,не более 2-3,True
Цветность,градусы,б/цвета,не более 30,True
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
Нитраты (по NO3),мг/дм3,24,не более 45,True
Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
Хлориды (Cl),мг/дм3,200,не более 350,True


# Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?
В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

Для начала получим аналитическое решение:
Нам необходимо найти вероятность того, что 3 вытянутых фрукта окажутся апельсинами. Используем формулу всех элементарных событий $Р = \frac{А}{N}$.
N - всевозможные выборки 3-х фруктов из 9. Количество таких возможных вариантов можно посчитать по формуле сочетаний (ведь на месте первого фрукта может быть любой из 9, на месте второго фрукта - любой из оставшихся 8 и тд, при этом порядок не важен): $C_9^3=\frac{9!}{3!*6!}=84$.
А - такие выборки фруктов, что в них апельсинов 3 штуки и ни одного яблока. Значит такую комбинацию можно посчитать перемножив два события: (3 апельсина из 5) * (0 яблок из 4). $А=C_5^3*С_4^0=\frac{5!}{3!*2!}*\frac{4!}{0!*4!}=10$. Тогда $Р = \frac{А}{N}=\frac{10}{84}=0,119$.

In [6]:
# найдём решение алгоритмически
n = 100000 # число испытаний
fruits = ['apple'] * 4 + ['orange'] * 5 # все фрукты

# проводя n испытаний будем вытягивать 3 случайных фрукта и
# подсчитывать, когда все фрукты апельсины
cnt_all_orange = 0
for _ in range(n):
    combination = np.random.choice(fruits, size=3, replace=False)
    if set(combination) == {'orange'}: cnt_all_orange += 1

print(f'Полученная выроятность: {(cnt_all_orange / n):.3f}\nТеоретическая вероятность: 0,119')

Полученная выроятность: 0.118
Теоретическая вероятность: 0,119


Вероятности совпали!

# Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?
В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

Решим для начала аналитически:
Так как детали выбираются по одной до того момента, пока не попадётся стандартная, это означает, что порядок важен. Тогда нам нужно выбрать на первое место 1 из 3 нестандартных деталей, а на второе место уже 1 из 7 стандартных деталей (порядок выбора важен - по сути используем формулу размещений). Следовательно, таких комбинаций, удовлетворяющих условию 7 * 3 = 21. А всего комбинаций различного выбора 2 деталей из 10 (с учётом порядка) 10 * 9 = 90. Тогда $Р = \frac{А}{N}=\frac{21}{90}=0,233$.

In [7]:
# найдём решение алгоритмически
n = 200000 # число испытаний
details = ['standart'] * 7 + ['not_standart'] * 3 # все детали


# в каждом испытании будем выбирать новую деталь, до тех пор пока не встретим стандартную
# число проверок будет подсчитываться
# фактически нам нужно просто проверить сколько раз встречается комбинация [нестандарт, стандарт]
checks_with_second_try = 0

for _ in range(n):
    combination = np.random.choice(details, size=2, replace=False)
    if tuple(combination) == ('not_standart', 'standart'):
        checks_with_second_try += 1

print(f'Полученная выроятность: {(checks_with_second_try / n):.3f}\nТеоретическая вероятность: 0,233')

Полученная выроятность: 0.233
Теоретическая вероятность: 0,233
